In [2]:
import pandas as pd
import os

In [3]:
def read_file():
    full_data = pd.DataFrame()
    for file in os.listdir():
        if 'compiled' in file:
            data = pd.read_csv(file, index_col = 0)
            full_data = full_data.append(data)
    return full_data


In [5]:
data = read_file()

/var/folders/74/n55gd6gs38n5wj2k6hwrc_yc0000gn/T/ipykernel_83963/497571254.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append(data)
/var/folders/74/n55gd6gs38n5wj2k6hwrc_yc0000gn/T/ipykernel_83963/497571254.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append(data)
/var/folders/74/n55gd6gs38n5wj2k6hwrc_yc0000gn/T/ipykernel_83963/497571254.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append(data)
/var/folders/74/n55gd6gs38n5wj2k6hwrc_yc0000gn/T/ipykernel_83963/497571254.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append(da